#Import from Google Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Classes for the Training

#*Evaluation class*

In [2]:
#evual
import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn import metrics

class evaluate_model:

    def __init__(self):
        self.message = 'Running'

    def __repr__(self):
        return self.message

    @staticmethod
    def plot_loss_acc_curves(history):
        """Plot the loss and accuracy curves for training and validation data"""
        fig, ax = plt.subplots(2,1)
        ax[0].plot(history.history['loss'], color='b', label="Training loss")
        ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
        legend = ax[0].legend(loc='best', shadow=True)

        ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
        ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
        legend = ax[1].legend(loc='best', shadow=True)
        plt.show()

    @staticmethod
    def plot_confusion_matrix(cm, classes,
                              normalize=False,
                              title='Confusion matrix',
                              cmap=plt.cm.Blues):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=90)
        plt.yticks(tick_marks, classes)

        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.show()

    @staticmethod
    def Clf_report(model,Ytrue,YPred):
        print("Classification report for classifier %s:\n%s\n"
              % (model, metrics.classification_report(Ytrue, YPred)))


#Preprocessing Class

In [3]:
#process_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os,cv2

class Img_dataset:

    def __init__(self,Shape,Channel):
        self.shape = Shape
        self.channel = Channel


    @staticmethod
    def get_imgInfo(file_dir,have_group = False):

        if have_group :
            Categories = os.listdir(file_dir)
            for category in Categories:
                print('{} {} images'.format(category, len(os.listdir(os.path.join(file_dir,category)))))
            #Load the data to dataframe
            data=[]
            for category_id, category in enumerate(Categories):
                for imgfile in os.listdir(os.path.join(file_dir,category)):
                    data.append(['{}/{}/{}'.format(file_dir,category,imgfile),category_id,category])
            column = ['img_path','category_id','category']
            print("Note --> categories available !")
        else:
            data = []
            for imgfile in os.listdir(file_dir):
                data.append(['{}/{}'.format(file_dir,imgfile),imgfile])
            column = ['img_path','img']
        return pd.DataFrame(data,columns=column)


    def disp_group_image(self,data,no_img=6):
         #if there is 'category' inside the dataframe columns
         #np.any(axis=none) : at least 1 true appear, return true
        if np.any(data.columns == 'category'):
             group = data['category'].unique()
             Num_group = len(group)
             fig = plt.figure(1, figsize=(Num_group,Num_group))
             grid = ImageGrid(fig, 111, nrows_ncols=(Num_group, Num_group), axes_pad=0.05)
             i2 = -1
             for grp in group:
                 img_path = np.array(data[data['category'] == grp]['img_path'])
                 for i,imgdir in enumerate(img_path):
                     if i < Num_group:
                         i2 = i2 + 1
                         img = cv2.imread(imgdir)
                         img = cv2.resize(img,dsize =(self.shape,self.shape),interpolation = cv2.INTER_AREA)
                         ax = grid[i2] #Put img into grid with specific position
                         ax.imshow(img)
                         ax.axis('off')
                     else:
                         #Display each group name across each row
                         ax.text(self.shape+30, self.shape/2, grp, verticalalignment='center')
                         break
        else:
             i2 = -1
             fig = plt.figure(1, figsize=(no_img,no_img))
             grid = ImageGrid(fig, 111, nrows_ncols=(no_img, no_img), axes_pad=0.05)
             img_path = np.array(data['img_path'])
             for i,imgdir in enumerate(img_path):
                if i < no_img*no_img:
                    i2 = i2 + 1
                    img = cv2.imread(imgdir)
                    img = cv2.resize(img,dsize =(self.shape,self.shape),interpolation = cv2.INTER_AREA)
                    ax = grid[i2] #Put img into grid with specific position
                    ax.imshow(img)
                    ax.axis('off')
                else:
                    break
        plt.show()


    def load_image(self,data,extract = False,lwr_hsv=None,upr_hsv=None):
        img_data = []
        img_path = data['img_path']
        getImg = True
        for imgdir in img_path:
            img = cv2.imread(imgdir)

            if extract :
                if lwr_hsv and upr_hsv :
                    blurr = cv2.GaussianBlur(img,(5,5),0)
                    hsv = cv2.cvtColor(blurr,cv2.COLOR_BGR2HSV)
                    mask = cv2.inRange(hsv,lwr_hsv,upr_hsv)
                    struc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
                    mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,struc)
                    boolean = mask>0
                    new_img = np.zeros_like(img,np.uint8)
                    new_img[boolean] = img[boolean]
                    res_img = new_img
                    if getImg:
                        plt.subplot(2,3,1);plt.imshow(img)# ORIGINAL
                        plt.subplot(2,3,2);plt.imshow(blurr) # BLURRED
                        plt.subplot(2,3,3);plt.imshow(hsv) # HSV CONVERTED
                        plt.subplot(2,3,4);plt.imshow(mask) # MASKED
                        plt.subplot(2,3,5);plt.imshow(boolean) # BOOLEAN MASKED
                        plt.subplot(2,3,6);plt.imshow(new_img)# NEW PROCESSED IMAGE
                        plt.axis('off')
                        plt.show()
                        getImg = False
                else:
                    raise ValueError('lwr_hsv or upr_hsv arguments are empty !')

            else:
                blurr = cv2.GaussianBlur(img,(5,5),0)
                rgb = cv2.cvtColor(blurr,cv2.COLOR_BGR2RGB)
                res_img = rgb

            rez_img = cv2.resize(res_img,dsize =(self.shape,self.shape),interpolation = cv2.INTER_AREA)
            rez_img = rez_img.reshape(self.shape,self.shape,self.channel)
            img_data.append(rez_img)

        return img_data


# Tool Class

In [4]:
#tool
from datetime import datetime
import numpy as np
import zipfile
from shutil import unpack_archive
import os

class tools:

    @staticmethod
    def start_end_timer(start_time=None):
        if not start_time:
            start_time = datetime.now()
            return start_time
        elif start_time:
            thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
            tmin, tsec = divmod(temp_sec, 60)
            print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

    @staticmethod
    def data_size(data,format = 'B'):
        if not isinstance(data,np.ndarray):
            raise TypeError('It is not an array !')

        if format == 'B':
            format_bytes = 1
        elif format == 'MB':
            format_bytes = 2**20 # 1 Megabyte is 1,048,576 bytes
        elif format == 'GB':
            format_bytes = 2**30 # 1 Gigabyte is 1,073,741,824 bytes
        else:
            raise Exception('choose B, MB & GB only !')


        total = data.nbytes/format_bytes
        return '{:.2f} {}'.format(total,format)


    @staticmethod
    def check_zip_file(dir_zip):
        with zipfile.ZipFile(dir_zip) as z:
          for filename in z.namelist():
                  print(filename)

    @staticmethod
    def unpack_zip_file(dir_zip,des_dir):
        for zdir in dir_zip:
          unpack_archive(zdir,des_dir)


#Import Classes and settings for training

In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json


import numpy as np 
import pandas as pd
from glob import glob 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import ResNet152
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential



main_dir = '/content/drive/MyDrive/plant-seedlings-classification'
train_dir = main_dir +'/train'
test_dir = main_dir + '/test'

size = 80
channel = 3
img_data = Img_dataset(size,channel)
Num_classes = len(['Black-grass','Charlock','Cleavers','Common Chickweed','Common wheat','Fat Hen','Loose Silky-bent','Maize'
 ,'Scentless Mayweed','Shepherds Purse','Small-flowered Cranesbill','Sugar beet'])

# Preprocossing 

In [ ]:
""" Loading Train Data """
train_data = img_data.get_imgInfo(train_dir,have_group = True)
train_data.head()


""" Loading Test Data """
test_data = img_data.get_imgInfo(test_dir)
test_data.head()


""" Display an Image in Grid"""
print('\n')
print('---train_data image grid---')
img_data.disp_group_image(train_data)
print('---test_data image grid---')
img_data.disp_group_image(test_data,6)


""" Load Train and Test Data """
lower = (25,40,50)
upper = (75,255,255)
print('\n')
print('---train_data image-processing---')
load_xtrain = img_data.load_image(train_data,extract=True,lwr_hsv=lower,upr_hsv=upper)
print('---test_data image-processing---')
load_xtest = img_data.load_image(test_data,extract=True,lwr_hsv=lower,upr_hsv=upper)

xtrain = np.array(load_xtrain)
xtest = np.array(load_xtest)
print('\n')
print('Orignal xtrain shape : {}'.format(xtrain.shape)) #(4750, dim, dim, 3)
print('Orignal xtest shape : {}'.format(xtest.shape))


"""Normalization"""
#Pixels are represented in the range [0-255],
#but the NN converges faster with smaller values, in the range [0-1].
xtrain = xtrain/255.0
xtest = xtest/255.0


"""Label encoding target variable """
ytrain = train_data.iloc[:,2]
species = ytrain.unique()
labels = preprocessing.LabelEncoder()
labels.fit(species)
encodedlabels = labels.transform(ytrain)
print('\n')
print('Classes'+str(labels.classes_))


""" One Hot Encoding """
ytrain = to_categorical(encodedlabels)


""" Check data memory used"""
print('\n')
print('Train data size : {}'.format(tools.data_size(xtrain,'GB')))
print(' Test data size : {}'.format(tools.data_size(xtest,'GB')))


"""Split training and validation sets"""
#stratify (Classification only) :
#It in train_test_split ensures that there is no overrepresentation of classes in the val set.
#It is used to avoid some labels being overrepresented in the val set.
#Note: only works with sklearn version > 0.17
xtrain, xval, ytrain, yval = train_test_split(xtrain,ytrain, test_size=0.1,random_state=2,stratify=ytrain) # 0.1 = 10%
print('\n---Split training and validation sets---')
print('xtrain shape : {}'.format(xtrain.shape))
print('ytrain shape : {}'.format(ytrain.shape))
print('  xval shape : {}'.format(xval.shape))
print('  yval shape : {}'.format(yval.shape))


Maize 221 images
Fat Hen 475 images
Loose Silky-bent 654 images
Common Chickweed 611 images
Sugar beet 385 images
Common wheat 221 images
Scentless Mayweed 516 images
Small-flowered Cranesbill 496 images
Shepherds Purse 231 images
Cleavers 287 images
Charlock 390 images
Black-grass 263 images
Note --> categories available !


---train_data image grid---


#Training (Transfer Learning Models)

**VGG16**


In [ ]:


# Create a VGG16 model
vgg = VGG16(input_shape=(size, size, channel ), weights='imagenet', include_top=False) #Training with Imagenet weights


#train the layers with custom data, these two lines can be ommitted. 
for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output) #Output obtained on vgg16 is now flattened. 
prediction = Dense( Num_classes, activation='softmax')(x) 

#Creating model object 
model = Model(inputs=vgg.input, outputs=prediction)

model.summary()


58892288/58889256 [==============================] - 0s 0us/step
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 80, 80, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 80, 80, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 80, 80, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 40, 40, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 40, 40, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 40, 40, 128)       147584    
____________________________________________________________

In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

Epoch 1/120
33/33 - 300s - loss: 1.9684 - accuracy: 0.3593 - val_loss: 1.4830 - val_accuracy: 0.5684
Epoch 2/120


KeyboardInterrupt: ignored

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_VGG16.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#VGG19


In [ ]:


# Create a VGG19 model
vgg = VGG19(input_shape=(size, size, channel ), weights='imagenet', include_top=False) 

#train the layers with custom data, these two lines can be ommitted. 
for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output) #Output obtained on vgg19 is now flattened. 
prediction = Dense( Num_classes, activation='softmax')(x) 

#Creating model object 
model = Model(inputs=vgg.input, outputs=prediction)

model.summary()


In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_VGG19.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#InceptionV3


In [ ]:


# Create a InceptionV3 model
InceptionV3 = applications.InceptionV3(input_shape=(size, size, channel )
    ,include_top=False, weights='imagenet' )



x = Flatten()(InceptionV3.output) #Output obtained on InceptionV3 is now flattened. 
prediction = Dense( Num_classes, activation='softmax')(x) 

#Creating model object 
model = Model(inputs=InceptionV3.input, outputs=prediction)

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 39, 39, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 39, 39, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 39, 39, 32)   0           batch_normalization_94[0][0]     
______________________________________________________________________________________________

In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_InceptionV3.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#ResNet50

In [ ]:


# Create a ResNet50 model
ResNet50 = ResNet50(
    include_top=False, weights='imagenet',
    input_shape=(size, size, channel ))



x = Flatten()(ResNet50.output) #Output obtained on ResNet50 is now flattened. 
prediction = Dense( Num_classes, activation='softmax')(x) 

#Creating model object 
model = Model(inputs=ResNet50.input, outputs=prediction)

model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 86, 86, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 40, 40, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 40, 40, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_ResNet50.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#ResNet101

In [ ]:


# Create a ResNet101 model
ResNet101 = ResNet101(
    include_top=False, weights='imagenet',
    input_shape=(size, size, channel ))



x = Flatten()(ResNet101.output) #Output obtained on ResNet101 is now flattened. 
prediction = Dense( Num_classes, activation='softmax')(x) 

#Creating model object 
model = Model(inputs=ResNet101.input, outputs=prediction)

model.summary()



171450368/171446536 [==============================] - 2s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 86, 86, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 40, 40, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 40, 40, 64)   256         conv1_conv[0][0]                 
_________________________

In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_ResNet101.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#ResNet152

In [ ]:


# Create a ResNet152 model
ResNet152 = ResNet152(
    include_top=False, weights='imagenet',
    input_shape=(size, size, channel ))



x = Flatten()(ResNet152.output) #Output obtained on ResNet152 is now flattened. 
prediction = Dense( Num_classes, activation='softmax')(x) 

#Creating model object 
model = Model(inputs=ResNet152.input, outputs=prediction)

model.summary()



234700800/234698864 [==============================] - 2s 0us/step
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 86, 86, 3)    0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 40, 40, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 40, 40, 64)   256         conv1_conv[0][0]                 
_________________________

In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_ResNet152.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#Transfer learning with additional Dense Layers with batch normalization

#VGG16


In [ ]:

# Create a VGG16 model
vgg = VGG16(input_shape=(size, size, channel ), weights='imagenet', include_top=False) #Training with Imagenet weights

model= Sequential()
model.add(vgg) 
model.add(Flatten()) 


#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=512))#VGG16
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(512,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(256,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(128,activation=('relu')))
model.add(BatchNormalization())
model.add(Dropout(.5))

model.add(Dense(Num_classes,activation=('softmax'))) 

#Checking the final model summary
model.summary()


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_25 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_385 (Bat (None, 1024)              4096      
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               524800    
_________________________________________________________________
batch_normalization_386 (Bat (None, 512)             

In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 1
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

KeyboardInterrupt: ignored

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_VGG16_AdditionDenseLayers.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

VGG19


In [ ]:


# Create a VGG19 model
vgg = VGG19(input_shape=(size, size, channel ), weights='imagenet', include_top=False) 
model= Sequential()
model.add(vgg) 
model.add(Flatten()) 


#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=512))#VGG19
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(512,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(256,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(128,activation=('relu')))
model.add(BatchNormalization())
model.add(Dropout(.5))

model.add(Dense(Num_classes,activation=('softmax'))) 

#Checking the final model summary
model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_17 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 12)                24588     
Total params: 20,048,972
Trainable params: 20,048,972
Non-trainable params: 0
_________________________________________________________________


In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_VGG19_AdditionDenseLayers.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#InceptionV3


In [ ]:
# Create a InceptionV3 model
InceptionV3 = applications.InceptionV3(input_shape=(size, size, channel )
    ,include_top=False, weights='imagenet' )

model= Sequential()
model.add(InceptionV3) 
model.add(Flatten()) 


#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=2048))#inception_v3 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(512,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(256,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(128,activation=('relu')))
model.add(BatchNormalization())
model.add(Dropout(.5))

model.add(Dense(Num_classes,activation=('softmax'))) 

#Checking the final model summary
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 1, 1, 2048)        21802784  
_________________________________________________________________
flatten_13 (Flatten)         (None, 2048)              0         
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_InceptionV3_AdditionDenseLayers.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#ResNet50

In [ ]:
# Create a ResNet50 model
ResNet50 = ResNet50(
    include_top=False, weights='imagenet',
    input_shape=(size, size, channel ))

model= Sequential()
model.add(ResNet50) 
model.add(Flatten()) 


#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=2048))#ResNet50 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(512,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(256,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(128,activation=('relu')))
model.add(BatchNormalization())
model.add(Dropout(.5))

model.add(Dense(Num_classes,activation=('softmax'))) 

#Checking the final model summary
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 3, 3, 2048)        23587712  
_________________________________________________________________
flatten_14 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 12)                221196    
Total params: 23,808,908
Trainable params: 23,755,788
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission


In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_ResNet50_AdditionDenseLayers.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

ResNet101

In [ ]:
# Create a ResNet50 model
ResNet101 = ResNet101(
    include_top=False, weights='imagenet',
    input_shape=(size, size, channel ))

model= Sequential()
model.add(ResNet101) 
model.add(Flatten()) 


#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=2048))#ResNet101 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(512,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(256,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(128,activation=('relu')))
model.add(BatchNormalization())
model.add(Dropout(.5))

model.add(Dense(Num_classes,activation=('softmax'))) 

#Checking the final model summary
model.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 3, 3, 2048)        42658176  
_________________________________________________________________
flatten_15 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 12)                221196    
Total params: 42,879,372
Trainable params: 42,774,028
Non-trainable params: 105,344
_________________________________________________________________


In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission



In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_ResNet101_AdditionDenseLayers.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)

#ResNet152

In [ ]:
# Create a ResNet50 model
ResNet152 = ResNet152(
    include_top=False, weights='imagenet',
    input_shape=(size, size, channel ))

model= Sequential()
model.add(ResNet152) 
model.add(Flatten()) 


#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=2048))#ResNet152 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(512,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(256,activation=('relu'))) 
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(128,activation=('relu')))
model.add(BatchNormalization())
model.add(Dropout(.5))

model.add(Dense(Num_classes,activation=('softmax'))) 

#Checking the final model summary
model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 3, 3, 2048)        58370944  
_________________________________________________________________
flatten_16 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 12)                221196    
Total params: 58,592,140
Trainable params: 58,440,716
Non-trainable params: 151,424
_________________________________________________________________


In [ ]:

start_time = tools.start_end_timer(None)
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics=["accuracy"])


#Data augmentation
#Alter the training data with small transformations
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )

datagen.fit(xtrain)

#Training
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=6,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit Model
epochs = 120
batch_size = 128

history = model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),
                              epochs = epochs, validation_data = (xval,yval),
                              verbose = 2, steps_per_epoch=xtrain.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])


tools.start_end_timer(start_time)


#Evaluate
start_time = tools.start_end_timer(None)
Y_pred = model.predict(xval)
tools.start_end_timer(start_time)

#Return the column position where the max value is
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(yval,axis = 1)

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
evaluate_model.plot_confusion_matrix(confusion_mtx, classes = species)
evaluate_model.plot_loss_acc_curves(history)
evaluate_model.Clf_report(model,Y_true,Y_pred_classes)

#Prediction use for Kaggle Submission



In [ ]:
y_test = model.predict(xtest)


submit_filename = '/content/drive/MyDrive/plant-seedlings-classification/PlantSeed_Classification_ResNet101_AdditionDenseLayers.csv'
Y_pred_classes = np.argmax(y_test,axis = 1)
Y_pred_df = labels.inverse_transform(Y_pred_classes)
results_data = pd.DataFrame(data={'file':test_data['img'], 'species':Y_pred_df})
results_data.to_csv(submit_filename, index=False)